In [130]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
import tensorflow as tf
import time

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

In [131]:
# load data: digits 5 to 9, but still label with 0 to 4, 
# because TensorFlow expects label's integers from 0 to n_classes-1.
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

X_train2_full = mnist.train.images[mnist.train.labels >= 5]
y_train2_full = mnist.train.labels[mnist.train.labels >= 5] - 5
X_valid2_full = mnist.validation.images[mnist.validation.labels >= 5]
y_valid2_full = mnist.validation.labels[mnist.validation.labels >= 5] - 5
X_test2 = mnist.test.images[mnist.test.labels >= 5]
y_test2 = mnist.test.labels[mnist.test.labels >= 5] - 5

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [132]:
# we want to keep only 100 instances per class in the training set 
# and let's keep only 30 instances per class in the validation set
# tesing set is already loaded above
def sample_n_instances_per_class(X, y, n=100):
    Xs, ys = [], []
    for label in np.unique(y):
        idx = (y == label)
        Xc = X[idx][:n]
        yc = y[idx][:n]
        Xs.append(Xc)
        ys.append(yc)
    return np.concatenate(Xs), np.concatenate(ys)

X_train2, y_train2 = sample_n_instances_per_class(X_train2_full, y_train2_full, n=100)
X_valid2, y_valid2 = sample_n_instances_per_class(X_valid2_full, y_valid2_full, n=30)

# HW 3.1 Only Softmax trainable for predict 5 to 9

In [136]:
reset_graph()

# For AdamOptimizer
learning_rate = 0.001

# import model from HW2
restore_saver = tf.train.import_meta_graph("./hw2_model/Team20_HW2.ckpt.meta")

# Step1: Get tensor from HW2 model
x = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("Y:0")
loss = tf.get_default_graph().get_tensor_by_name("loss:0")
accuracy = tf.get_default_graph().get_tensor_by_name("accuracy:0")
training_mode = tf.get_default_graph().get_tensor_by_name("is_training:0")

# Step2: Get the outputs of our softmax layer, see tensorboard to know the output of logits tensor is logits/BiasAdd.
logits = tf.get_default_graph().get_tensor_by_name("logits/BiasAdd:0")

# Step3: Only let softmax layer trainable
output_layer = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='logits')
optimizer = tf.train.AdamOptimizer(learning_rate, name='opt1')
training_op = optimizer.minimize(loss, var_list=output_layer)

print(logits)
print(output_layer)

Tensor("logits/BiasAdd:0", shape=(?, 5), dtype=float32)
[<tf.Variable 'logits/kernel:0' shape=(128, 5) dtype=float32_ref>, <tf.Variable 'logits/bias:0' shape=(5,) dtype=float32_ref>]


In [137]:
# To calculate prediction result and accuracy
prediction = tf.argmax(logits, 1, output_type=tf.int32)
correct_prediction = tf.equal(prediction, tf.cast(y, tf.int32))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')

In [101]:
# For initialize
init_g = tf.global_variables_initializer()

# Saver
saver = tf.train.Saver()

In [105]:
# some hyperparams
batch_size = 128
# An epoch means one iteration over all of the training data
train_steps = round(len(X_train2) / batch_size)

training_epochs = 1000
# If validation loss does not improve after certain steps of training, apply early stopping
early_stopping_epoch = 20

# Early stopping variables
best_loss = 10000000
best_epoch = 0
early_stopped = False

# Prepare our training dataset with batch
dataset_batch = tf.contrib.data.Dataset.from_tensor_slices((X_train2, y_train2)).batch(batch_size).repeat(training_epochs)
dataset_batch = dataset_batch.make_initializable_iterator()


with tf.Session() as sess:
    # initialize
    sess.run(init_g)
    sess.run(dataset_batch.initializer)
    
    # restore our model
    restore_saver.restore(sess, "./hw2_model/Team20_HW2.ckpt")
    for var in output_layer:
        var.initializer.run()

    start_time = time.time()
    
    # Training 1000 epochs
    for epoch in range(0, training_epochs):
        # Training steps
        for i in range(train_steps):
            X_in, y_in = sess.run(dataset_batch.get_next())
            sess.run(training_op, feed_dict={x: X_in, y: y_in, training_mode: False})

        # Validate accuracy every epoch
        curr_loss, curr_accuracy = sess.run([loss, accuracy], feed_dict={x: X_valid2, y: y_valid2, training_mode: False})
        
        # Save checkpoint of current model if it performs better
        if best_loss > curr_loss:
            best_loss = curr_loss
            save_path = saver.save(sess, "./Team20_HW3.ckpt")
            best_epoch = epoch        
        # Early stop if model does not improve for certain epoch
        elif epoch - best_epoch >= early_stopping_epoch:
            early_stopped = True
            break
        
        print("Epoch {}: Validation loss: {} Best loss: {} Accuracy: {} ".format(epoch, curr_loss, best_loss ,curr_accuracy))

    # Save checkpoint in case the training is not early-stopped
    if not early_stopped:
        print("save best model")
        save_path = saver.save(sess, "./Team20_HW3.ckpt")

    end_time = time.time()
    print("Total training time: " + str(end_time - start_time))    
    
    # Get the best model
    saver.restore(sess, "./Team20_HW3.ckpt")
    
    # Total accuracy
    final_accuracy = sess.run(accuracy, feed_dict={x: X_test2, y: y_test2, training_mode: False})
    print("Test accuracy: ", final_accuracy)


INFO:tensorflow:Restoring parameters from ./hw2_model/Team20_HW2.ckpt
Epoch 0: Validation loss: 4.056200981140137 Best loss: 4.056200981140137 Accuracy: 0.2666666507720947 
Epoch 1: Validation loss: 3.7506089210510254 Best loss: 3.7506089210510254 Accuracy: 0.273333340883255 
Epoch 2: Validation loss: 3.4611799716949463 Best loss: 3.4611799716949463 Accuracy: 0.29333335161209106 
Epoch 3: Validation loss: 3.190791606903076 Best loss: 3.190791606903076 Accuracy: 0.30666667222976685 
Epoch 4: Validation loss: 2.940844774246216 Best loss: 2.940844774246216 Accuracy: 0.3333333134651184 
Epoch 5: Validation loss: 2.71216082572937 Best loss: 2.71216082572937 Accuracy: 0.3399999737739563 
Epoch 6: Validation loss: 2.5051674842834473 Best loss: 2.5051674842834473 Accuracy: 0.36666664481163025 
Epoch 7: Validation loss: 2.319786787033081 Best loss: 2.319786787033081 Accuracy: 0.42666664719581604 
Epoch 8: Validation loss: 2.1552441120147705 Best loss: 2.1552441120147705 Accuracy: 0.439999997615

Epoch 78: Validation loss: 0.6108630299568176 Best loss: 0.6108630299568176 Accuracy: 0.7999999523162842 
Epoch 79: Validation loss: 0.6083198189735413 Best loss: 0.6083198189735413 Accuracy: 0.7999999523162842 
Epoch 80: Validation loss: 0.6058343648910522 Best loss: 0.6058343648910522 Accuracy: 0.8066666126251221 
Epoch 81: Validation loss: 0.6034047603607178 Best loss: 0.6034047603607178 Accuracy: 0.8066666126251221 
Epoch 82: Validation loss: 0.6010286211967468 Best loss: 0.6010286211967468 Accuracy: 0.8199999928474426 
Epoch 83: Validation loss: 0.5987042188644409 Best loss: 0.5987042188644409 Accuracy: 0.8199999928474426 
Epoch 84: Validation loss: 0.5964294672012329 Best loss: 0.5964294672012329 Accuracy: 0.8199999928474426 
Epoch 85: Validation loss: 0.5942028760910034 Best loss: 0.5942028760910034 Accuracy: 0.8199999928474426 
Epoch 86: Validation loss: 0.5920225381851196 Best loss: 0.5920225381851196 Accuracy: 0.8199999928474426 
Epoch 87: Validation loss: 0.5898870229721069 

Epoch 156: Validation loss: 0.5050200819969177 Best loss: 0.5050200819969177 Accuracy: 0.846666693687439 
Epoch 157: Validation loss: 0.5043348073959351 Best loss: 0.5043348073959351 Accuracy: 0.8466666340827942 
Epoch 158: Validation loss: 0.5036590695381165 Best loss: 0.5036590695381165 Accuracy: 0.846666693687439 
Epoch 159: Validation loss: 0.5029927492141724 Best loss: 0.5029927492141724 Accuracy: 0.846666693687439 
Epoch 160: Validation loss: 0.5023357272148132 Best loss: 0.5023357272148132 Accuracy: 0.846666693687439 
Epoch 161: Validation loss: 0.5016879439353943 Best loss: 0.5016879439353943 Accuracy: 0.846666693687439 
Epoch 162: Validation loss: 0.5010491609573364 Best loss: 0.5010491609573364 Accuracy: 0.846666693687439 
Epoch 163: Validation loss: 0.5004191994667053 Best loss: 0.5004191994667053 Accuracy: 0.846666693687439 
Epoch 164: Validation loss: 0.49979811906814575 Best loss: 0.49979811906814575 Accuracy: 0.846666693687439 
Epoch 165: Validation loss: 0.4991855621337

Epoch 233: Validation loss: 0.47260624170303345 Best loss: 0.47260624170303345 Accuracy: 0.8466666340827942 
Epoch 234: Validation loss: 0.4723809063434601 Best loss: 0.4723809063434601 Accuracy: 0.8466666340827942 
Epoch 235: Validation loss: 0.4721592366695404 Best loss: 0.4721592366695404 Accuracy: 0.8466666340827942 
Epoch 236: Validation loss: 0.47194114327430725 Best loss: 0.47194114327430725 Accuracy: 0.8466666340827942 
Epoch 237: Validation loss: 0.47172653675079346 Best loss: 0.47172653675079346 Accuracy: 0.8466666340827942 
Epoch 238: Validation loss: 0.47151532769203186 Best loss: 0.47151532769203186 Accuracy: 0.8466666340827942 
Epoch 239: Validation loss: 0.471307635307312 Best loss: 0.471307635307312 Accuracy: 0.8466666340827942 
Epoch 240: Validation loss: 0.4711032509803772 Best loss: 0.4711032509803772 Accuracy: 0.8466666340827942 
Epoch 241: Validation loss: 0.47090232372283936 Best loss: 0.47090232372283936 Accuracy: 0.8466666340827942 
Epoch 242: Validation loss: 0

Epoch 310: Validation loss: 0.46345725655555725 Best loss: 0.46345725655555725 Accuracy: 0.8399999737739563 
Epoch 311: Validation loss: 0.4634222686290741 Best loss: 0.4634222686290741 Accuracy: 0.8399999737739563 
Epoch 312: Validation loss: 0.46338894963264465 Best loss: 0.46338894963264465 Accuracy: 0.8399999737739563 
Epoch 313: Validation loss: 0.46335726976394653 Best loss: 0.46335726976394653 Accuracy: 0.8400000333786011 
Epoch 314: Validation loss: 0.4633270502090454 Best loss: 0.4633270502090454 Accuracy: 0.8399999737739563 
Epoch 315: Validation loss: 0.4632984399795532 Best loss: 0.4632984399795532 Accuracy: 0.8399999737739563 
Epoch 316: Validation loss: 0.4632714092731476 Best loss: 0.4632714092731476 Accuracy: 0.8399999737739563 
Epoch 317: Validation loss: 0.46324580907821655 Best loss: 0.46324580907821655 Accuracy: 0.8399999737739563 
Epoch 318: Validation loss: 0.46322181820869446 Best loss: 0.46322181820869446 Accuracy: 0.846666693687439 
Epoch 319: Validation loss: 

# HW 3.2 Cache 5th layer

In [99]:
# Step 3.5
# Retrive the output of fifth layer, see tensorboard to know the output of fifth layer tensor is dense4/Elu.
fifth_dense_output = tf.get_default_graph().get_tensor_by_name("dense4/Elu:0")
print(fifth_dense_output)

Tensor("dense4/Elu:0", shape=(?, 128), dtype=float32)


In [104]:
# some hyperparams
batch_size = 128
# An epoch means one iteration over all of the training data
train_steps = round(len(X_train2) / batch_size)

training_epochs = 1000
# If validation loss does not improve after certain steps of training, apply early stopping
early_stopping_epoch = 20

# Early stopping variables
best_loss = 10000000
best_epoch = 0
early_stopped = False

# Prepare our training dataset with batch
dataset_batch = tf.contrib.data.Dataset.from_tensor_slices((X_train2, y_train2)).batch(batch_size).repeat(training_epochs)
dataset_batch = dataset_batch.make_initializable_iterator()


with tf.Session() as sess:
    # initialize
    sess.run(init_g)
    sess.run(dataset_batch.initializer)
    
    # restore our model
    restore_saver.restore(sess, "./hw2_model/Team20_HW2.ckpt")
    for var in output_layer:
        var.initializer.run()
    
    start_time = time.time()
    
    # Feed validation set into 5th layer and save the parms
    dense5_output_valid = fifth_dense_output.eval(feed_dict={x: X_valid2, y: y_valid2, training_mode: False})
    
    # Training 1000 epochs
    for epoch in range(0, training_epochs):
        # Training steps
        for i in range(1, train_steps+1):
            X_in, y_in = sess.run(dataset_batch.get_next())
            # Feed training set into 5th layer per batch and save the parms
            dense5_output_train = fifth_dense_output.eval(feed_dict={x: X_in, y: y_in, training_mode: False})
            # Directly feed the params of dense5 for training
            sess.run(training_op, feed_dict={fifth_dense_output: dense5_output_train,  y: y_in, training_mode: False})

        # Validate accuracy every epoch
        curr_loss, curr_accuracy = sess.run([loss, accuracy], feed_dict={fifth_dense_output: dense5_output_valid, y: y_valid2, training_mode: False})
        
        # Save checkpoint of current model if it performs better
        if best_loss > curr_loss:
            best_loss = curr_loss
            save_path = saver.save(sess, "./Team20_HW3_2_cache_5_layer.ckpt")
            best_epoch = epoch        
        # Early stop if model does not improve for certain epoch
        elif epoch - best_epoch >= early_stopping_epoch:
            early_stopped = True
            break
        
        print("Epoch {}: Validation loss: {} Best loss: {} Accuracy: {} ".format(epoch, curr_loss, best_loss ,curr_accuracy))
    
    # Save checkpoint in case the training is not early-stopped
    if not early_stopped:
        print("save best model")
        save_path = saver.save(sess, "./Team20_HW3_2_cache_5_layer.ckpt")
    
    end_time = time.time()
    print("Total training time: " + str(end_time - start_time))

    # Get the best model
    saver.restore(sess, "./Team20_HW3_2_cache_5_layer.ckpt")
    
    # Total accuracy
    final_accuracy = sess.run(accuracy, feed_dict={x: X_test2, y: y_test2, training_mode: False})
    print("Test accuracy: ", final_accuracy)


INFO:tensorflow:Restoring parameters from ./hw2_model/Team20_HW2.ckpt
Epoch 0: Validation loss: 4.056200981140137 Best loss: 4.056200981140137 Accuracy: 0.2666666507720947 
Epoch 1: Validation loss: 3.7506086826324463 Best loss: 3.7506086826324463 Accuracy: 0.273333340883255 
Epoch 2: Validation loss: 3.4611799716949463 Best loss: 3.4611799716949463 Accuracy: 0.2933333218097687 
Epoch 3: Validation loss: 3.1907920837402344 Best loss: 3.1907920837402344 Accuracy: 0.30666667222976685 
Epoch 4: Validation loss: 2.940844774246216 Best loss: 2.940844774246216 Accuracy: 0.3333333134651184 
Epoch 5: Validation loss: 2.71216082572937 Best loss: 2.71216082572937 Accuracy: 0.3399999737739563 
Epoch 6: Validation loss: 2.5051674842834473 Best loss: 2.5051674842834473 Accuracy: 0.36666664481163025 
Epoch 7: Validation loss: 2.319786787033081 Best loss: 2.319786787033081 Accuracy: 0.4266666769981384 
Epoch 8: Validation loss: 2.1552441120147705 Best loss: 2.1552441120147705 Accuracy: 0.439999997615

Epoch 78: Validation loss: 0.6108630299568176 Best loss: 0.6108630299568176 Accuracy: 0.7999999523162842 
Epoch 79: Validation loss: 0.6083198189735413 Best loss: 0.6083198189735413 Accuracy: 0.7999999523162842 
Epoch 80: Validation loss: 0.6058343648910522 Best loss: 0.6058343648910522 Accuracy: 0.8066666722297668 
Epoch 81: Validation loss: 0.6034047603607178 Best loss: 0.6034047603607178 Accuracy: 0.8066666126251221 
Epoch 82: Validation loss: 0.6010286211967468 Best loss: 0.6010286211967468 Accuracy: 0.8199999928474426 
Epoch 83: Validation loss: 0.5987041592597961 Best loss: 0.5987041592597961 Accuracy: 0.8199999928474426 
Epoch 84: Validation loss: 0.5964294672012329 Best loss: 0.5964294672012329 Accuracy: 0.8199999928474426 
Epoch 85: Validation loss: 0.5942028760910034 Best loss: 0.5942028760910034 Accuracy: 0.8199999928474426 
Epoch 86: Validation loss: 0.5920226573944092 Best loss: 0.5920226573944092 Accuracy: 0.8199999928474426 
Epoch 87: Validation loss: 0.5898870229721069 

Epoch 156: Validation loss: 0.5050200819969177 Best loss: 0.5050200819969177 Accuracy: 0.846666693687439 
Epoch 157: Validation loss: 0.5043348073959351 Best loss: 0.5043348073959351 Accuracy: 0.846666693687439 
Epoch 158: Validation loss: 0.5036590695381165 Best loss: 0.5036590695381165 Accuracy: 0.846666693687439 
Epoch 159: Validation loss: 0.5029927492141724 Best loss: 0.5029927492141724 Accuracy: 0.8466666340827942 
Epoch 160: Validation loss: 0.5023357272148132 Best loss: 0.5023357272148132 Accuracy: 0.846666693687439 
Epoch 161: Validation loss: 0.5016879439353943 Best loss: 0.5016879439353943 Accuracy: 0.8466666340827942 
Epoch 162: Validation loss: 0.5010491609573364 Best loss: 0.5010491609573364 Accuracy: 0.846666693687439 
Epoch 163: Validation loss: 0.5004191994667053 Best loss: 0.5004191994667053 Accuracy: 0.8466666340827942 
Epoch 164: Validation loss: 0.499798059463501 Best loss: 0.499798059463501 Accuracy: 0.846666693687439 
Epoch 165: Validation loss: 0.499185502529144

Epoch 233: Validation loss: 0.4726063013076782 Best loss: 0.4726063013076782 Accuracy: 0.8466666340827942 
Epoch 234: Validation loss: 0.4723809063434601 Best loss: 0.4723809063434601 Accuracy: 0.8466666340827942 
Epoch 235: Validation loss: 0.4721592664718628 Best loss: 0.4721592664718628 Accuracy: 0.8466666340827942 
Epoch 236: Validation loss: 0.47194114327430725 Best loss: 0.47194114327430725 Accuracy: 0.8466666340827942 
Epoch 237: Validation loss: 0.47172650694847107 Best loss: 0.47172650694847107 Accuracy: 0.8466666340827942 
Epoch 238: Validation loss: 0.47151535749435425 Best loss: 0.47151535749435425 Accuracy: 0.8466666340827942 
Epoch 239: Validation loss: 0.471307635307312 Best loss: 0.471307635307312 Accuracy: 0.8466666340827942 
Epoch 240: Validation loss: 0.4711032509803772 Best loss: 0.4711032509803772 Accuracy: 0.8466666340827942 
Epoch 241: Validation loss: 0.47090235352516174 Best loss: 0.47090235352516174 Accuracy: 0.8466666340827942 
Epoch 242: Validation loss: 0.4

Epoch 310: Validation loss: 0.46345722675323486 Best loss: 0.46345722675323486 Accuracy: 0.8399999737739563 
Epoch 311: Validation loss: 0.4634222984313965 Best loss: 0.4634222984313965 Accuracy: 0.8399999737739563 
Epoch 312: Validation loss: 0.46338897943496704 Best loss: 0.46338897943496704 Accuracy: 0.8399999737739563 
Epoch 313: Validation loss: 0.46335726976394653 Best loss: 0.46335726976394653 Accuracy: 0.8399999737739563 
Epoch 314: Validation loss: 0.4633270800113678 Best loss: 0.4633270800113678 Accuracy: 0.8399999737739563 
Epoch 315: Validation loss: 0.4632984399795532 Best loss: 0.4632984399795532 Accuracy: 0.8399999737739563 
Epoch 316: Validation loss: 0.4632713794708252 Best loss: 0.4632713794708252 Accuracy: 0.8399999737739563 
Epoch 317: Validation loss: 0.46324580907821655 Best loss: 0.46324580907821655 Accuracy: 0.8399999737739563 
Epoch 318: Validation loss: 0.46322187781333923 Best loss: 0.46322187781333923 Accuracy: 0.846666693687439 
Epoch 319: Validation loss: 

#### The total training time of HW3.1 is 893.5655002593994.
#### The total training time of HW3.2 is 773.5620684623718.
### We can see HW3.2 faster than HW3.1!

# HW 3.3  Only use 4 layer to train NN and add new softmax layer


In [106]:
reset_graph()

# For AdamOptimizer
learning_rate = 0.001
n_classes = 5

# import model from HW2
restore_saver = tf.train.import_meta_graph("./hw2_model/Team20_HW2.ckpt.meta")

# Step1: Get tensor from HW2 model
x = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("Y:0")
training_mode = tf.get_default_graph().get_tensor_by_name("is_training:0")

# Step2: Retrive the output of fourth layer
fourth_dense_output = tf.get_default_graph().get_tensor_by_name("dense3/Elu:0")
print(fourth_dense_output)

# And Make a new softmax layers
logits = tf.layers.dense(inputs=fourth_dense_output, units=n_classes, kernel_initializer=
                                    tf.contrib.layers.variance_scaling_initializer(), name='logits_new')

cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits, name='sparse_cross_entropy')
loss = tf.reduce_mean(cross_entropy, name='loss')

# Step3: Only let softmax layer trainable
output_layer = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='logits_new')
optimizer = tf.train.AdamOptimizer(learning_rate, name='opt3_3')
training_op = optimizer.minimize(loss, var_list=output_layer)

print(logits)

Tensor("dense3/Elu:0", shape=(?, 128), dtype=float32)
Tensor("logits_new/BiasAdd:0", shape=(?, 5), dtype=float32)


In [107]:
prediction = tf.argmax(logits, 1, output_type=tf.int32)
correct_prediction = tf.equal(prediction, tf.cast(y, tf.int32))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')

In [108]:
# For initialize
init_g = tf.global_variables_initializer()

# Saver
saver = tf.train.Saver()

In [109]:
# some hyperparams
batch_size = 128
# An epoch means one iteration over all of the training data
train_steps = round(len(X_train2) / batch_size)

training_epochs = 1000
# If validation loss does not improve after certain steps of training, apply early stopping
early_stopping_epoch = 20

# Early stopping variables
best_loss = 10000000
best_epoch = 0
early_stopped = False

# Prepare our training dataset with batch
dataset_batch = tf.contrib.data.Dataset.from_tensor_slices((X_train2, y_train2)).batch(batch_size).repeat(training_epochs)
dataset_batch = dataset_batch.make_initializable_iterator()


with tf.Session() as sess:
    # initialize
    sess.run(init_g)
    sess.run(dataset_batch.initializer)
    
    # restore our model
    restore_saver.restore(sess, "./hw2_model/Team20_HW2.ckpt")
    for var in output_layer:
        var.initializer.run()

    start_time = time.time()
    
    # Training 1000 epochs
    for epoch in range(0, training_epochs):
        # Training steps
        for i in range(train_steps):
            X_in, y_in = sess.run(dataset_batch.get_next())
            sess.run(training_op, feed_dict={x: X_in, y: y_in, training_mode: False})

        # Validate accuracy every epoch
        curr_loss, curr_accuracy = sess.run([loss, accuracy], feed_dict={x: X_valid2, y: y_valid2, training_mode: False})
        
        # Save checkpoint of current model if it performs better
        if best_loss > curr_loss:
            best_loss = curr_loss
            save_path = saver.save(sess, "./Team20_HW3_3_four_layers.ckpt")
            best_epoch = epoch        
        # Early stop if model does not improve for certain epoch
        elif epoch - best_epoch >= early_stopping_epoch:
            early_stopped = True
            break
        
        print("Epoch {}: Validation loss: {} Best loss: {} Accuracy: {} ".format(epoch, curr_loss, best_loss ,curr_accuracy))

    # Save checkpoint in case the training is not early-stopped
    if not early_stopped:
        print("save best model")
        save_path = saver.save(sess, "./Team20_HW3_3_four_layers.ckpt")

    end_time = time.time()
    print("Total training time: " + str(end_time - start_time))    
    
    # Get the best model
    saver.restore(sess, "./Team20_HW3_3_four_layers.ckpt")
    
    # Total accuracy
    final_accuracy = sess.run(accuracy, feed_dict={x: X_test2, y: y_test2, training_mode: False})
    print("Test accuracy: ", final_accuracy)


INFO:tensorflow:Restoring parameters from ./hw2_model/Team20_HW2.ckpt
Epoch 0: Validation loss: 3.7095155715942383 Best loss: 3.7095155715942383 Accuracy: 0.14666667580604553 
Epoch 1: Validation loss: 3.4455342292785645 Best loss: 3.4455342292785645 Accuracy: 0.1666666567325592 
Epoch 2: Validation loss: 3.1958770751953125 Best loss: 3.1958770751953125 Accuracy: 0.19333332777023315 
Epoch 3: Validation loss: 2.9606239795684814 Best loss: 2.9606239795684814 Accuracy: 0.24000000953674316 
Epoch 4: Validation loss: 2.740607261657715 Best loss: 2.740607261657715 Accuracy: 0.25999999046325684 
Epoch 5: Validation loss: 2.5368072986602783 Best loss: 2.5368072986602783 Accuracy: 0.2866666615009308 
Epoch 6: Validation loss: 2.3499741554260254 Best loss: 2.3499741554260254 Accuracy: 0.36666664481163025 
Epoch 7: Validation loss: 2.1803154945373535 Best loss: 2.1803154945373535 Accuracy: 0.3866666555404663 
Epoch 8: Validation loss: 2.0273947715759277 Best loss: 2.0273947715759277 Accuracy: 0.

Epoch 78: Validation loss: 0.4785137474536896 Best loss: 0.4785137474536896 Accuracy: 0.8533332943916321 
Epoch 79: Validation loss: 0.47551119327545166 Best loss: 0.47551119327545166 Accuracy: 0.8533332943916321 
Epoch 80: Validation loss: 0.4725784361362457 Best loss: 0.4725784361362457 Accuracy: 0.8533332943916321 
Epoch 81: Validation loss: 0.469713419675827 Best loss: 0.469713419675827 Accuracy: 0.8466666340827942 
Epoch 82: Validation loss: 0.4669142961502075 Best loss: 0.4669142961502075 Accuracy: 0.8466666340827942 
Epoch 83: Validation loss: 0.4641791880130768 Best loss: 0.4641791880130768 Accuracy: 0.8466666340827942 
Epoch 84: Validation loss: 0.461506187915802 Best loss: 0.461506187915802 Accuracy: 0.8466666340827942 
Epoch 85: Validation loss: 0.45889371633529663 Best loss: 0.45889371633529663 Accuracy: 0.8466666340827942 
Epoch 86: Validation loss: 0.45633986592292786 Best loss: 0.45633986592292786 Accuracy: 0.8466666340827942 
Epoch 87: Validation loss: 0.453842967748641

Epoch 155: Validation loss: 0.35789942741394043 Best loss: 0.35789942741394043 Accuracy: 0.8800000548362732 
Epoch 156: Validation loss: 0.3570835292339325 Best loss: 0.3570835292339325 Accuracy: 0.8800000548362732 
Epoch 157: Validation loss: 0.3562777042388916 Best loss: 0.3562777042388916 Accuracy: 0.8800000548362732 
Epoch 158: Validation loss: 0.35548198223114014 Best loss: 0.35548198223114014 Accuracy: 0.8800000548362732 
Epoch 159: Validation loss: 0.3546959459781647 Best loss: 0.3546959459781647 Accuracy: 0.8800000548362732 
Epoch 160: Validation loss: 0.35391974449157715 Best loss: 0.35391974449157715 Accuracy: 0.8800000548362732 
Epoch 161: Validation loss: 0.3531529903411865 Best loss: 0.3531529903411865 Accuracy: 0.8800000548362732 
Epoch 162: Validation loss: 0.35239556431770325 Best loss: 0.35239556431770325 Accuracy: 0.8800000548362732 
Epoch 163: Validation loss: 0.35164737701416016 Best loss: 0.35164737701416016 Accuracy: 0.8800000548362732 
Epoch 164: Validation loss:

Epoch 231: Validation loss: 0.3163871765136719 Best loss: 0.3163871765136719 Accuracy: 0.8866667151451111 
Epoch 232: Validation loss: 0.3160334527492523 Best loss: 0.3160334527492523 Accuracy: 0.8866667151451111 
Epoch 233: Validation loss: 0.31568315625190735 Best loss: 0.31568315625190735 Accuracy: 0.8866667151451111 
Epoch 234: Validation loss: 0.3153362572193146 Best loss: 0.3153362572193146 Accuracy: 0.8866667151451111 
Epoch 235: Validation loss: 0.3149926960468292 Best loss: 0.3149926960468292 Accuracy: 0.8866667151451111 
Epoch 236: Validation loss: 0.3146525025367737 Best loss: 0.3146525025367737 Accuracy: 0.8866667151451111 
Epoch 237: Validation loss: 0.3143155574798584 Best loss: 0.3143155574798584 Accuracy: 0.8866667151451111 
Epoch 238: Validation loss: 0.31398189067840576 Best loss: 0.31398189067840576 Accuracy: 0.8866667151451111 
Epoch 239: Validation loss: 0.313651442527771 Best loss: 0.313651442527771 Accuracy: 0.8866667151451111 
Epoch 240: Validation loss: 0.31332

Epoch 308: Validation loss: 0.2970758378505707 Best loss: 0.2970758378505707 Accuracy: 0.8800000548362732 
Epoch 309: Validation loss: 0.29690951108932495 Best loss: 0.29690951108932495 Accuracy: 0.8800000548362732 
Epoch 310: Validation loss: 0.2967450022697449 Best loss: 0.2967450022697449 Accuracy: 0.8799999952316284 
Epoch 311: Validation loss: 0.29658228158950806 Best loss: 0.29658228158950806 Accuracy: 0.8800000548362732 
Epoch 312: Validation loss: 0.29642120003700256 Best loss: 0.29642120003700256 Accuracy: 0.8800000548362732 
Epoch 313: Validation loss: 0.2962619662284851 Best loss: 0.2962619662284851 Accuracy: 0.8800000548362732 
Epoch 314: Validation loss: 0.29610440135002136 Best loss: 0.29610440135002136 Accuracy: 0.8800000548362732 
Epoch 315: Validation loss: 0.29594865441322327 Best loss: 0.29594865441322327 Accuracy: 0.8800000548362732 
Epoch 316: Validation loss: 0.2957945466041565 Best loss: 0.2957945466041565 Accuracy: 0.8800000548362732 
Epoch 317: Validation loss:

Epoch 385: Validation loss: 0.2888084053993225 Best loss: 0.2888084053993225 Accuracy: 0.9000000357627869 
Epoch 386: Validation loss: 0.2887539863586426 Best loss: 0.2887539863586426 Accuracy: 0.9000000357627869 
Epoch 387: Validation loss: 0.288700670003891 Best loss: 0.288700670003891 Accuracy: 0.9000000357627869 
Epoch 388: Validation loss: 0.2886486053466797 Best loss: 0.2886486053466797 Accuracy: 0.9000000357627869 
Epoch 389: Validation loss: 0.2885977029800415 Best loss: 0.2885977029800415 Accuracy: 0.9000000357627869 
Epoch 390: Validation loss: 0.2885478436946869 Best loss: 0.2885478436946869 Accuracy: 0.9000000357627869 
Epoch 391: Validation loss: 0.2884993553161621 Best loss: 0.2884993553161621 Accuracy: 0.893333375453949 
Epoch 392: Validation loss: 0.28845182061195374 Best loss: 0.28845182061195374 Accuracy: 0.893333375453949 
Epoch 393: Validation loss: 0.28840553760528564 Best loss: 0.28840553760528564 Accuracy: 0.893333375453949 
Epoch 394: Validation loss: 0.28836035

#### Test accuracy of HW3.1 and HW3.2 are 0.821642
#### Test accuracy of HW3.3 is 0.851882
### Accuracy of HW3.3 is better than HW3.1&3.2

# HW 3.4  Bonus which extend HW3.3: only frozen third and fourth layers, and make first and second layers trainable


In [123]:
reset_graph()

# For AdamOptimizer
learning_rate = 0.001
n_classes = 5

# import model from HW2
restore_saver = tf.train.import_meta_graph("./hw2_model/Team20_HW2.ckpt.meta")

# Step1: Get tensor from HW2 model
x = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("Y:0")
training_mode = tf.get_default_graph().get_tensor_by_name("is_training:0")

# Retrive the output of fourth layer
fourth_dense_output = tf.get_default_graph().get_tensor_by_name("dense3/Elu:0")

# Make a new softmax layers
logits = tf.layers.dense(inputs=fourth_dense_output, units=n_classes, kernel_initializer=
                                    tf.contrib.layers.variance_scaling_initializer(), name='logits_new')

cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits, name='sparse_cross_entropy')
loss = tf.reduce_mean(cross_entropy, name='loss')

# make layer trainable
output_layer = []
# Make first layer trainable
output_layer.extend(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='dense0'))
# Make second layer trainable
output_layer.extend(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='dense1'))
# Make softmax layer trainable
output_layer.extend(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='logits_new'))

optimizer = tf.train.AdamOptimizer(learning_rate, name='opt3_4')
training_op = optimizer.minimize(loss, var_list=output_layer)


[<tf.Variable 'dense0/kernel:0' shape=(784, 128) dtype=float32_ref>, <tf.Variable 'dense0/bias:0' shape=(128,) dtype=float32_ref>, <tf.Variable 'dense1/kernel:0' shape=(128, 128) dtype=float32_ref>, <tf.Variable 'dense1/bias:0' shape=(128,) dtype=float32_ref>, <tf.Variable 'logits_new/kernel:0' shape=(128, 5) dtype=float32_ref>, <tf.Variable 'logits_new/bias:0' shape=(5,) dtype=float32_ref>]


In [125]:
prediction = tf.argmax(logits, 1, output_type=tf.int32)
correct_prediction = tf.equal(prediction, tf.cast(y, tf.int32))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')

In [126]:
# For initialize
init_g = tf.global_variables_initializer()

# Saver
saver = tf.train.Saver()

In [127]:
# some hyperparams
batch_size = 128
# An epoch means one iteration over all of the training data
train_steps = round(len(X_train2) / batch_size)

training_epochs = 1000
# If validation loss does not improve after certain steps of training, apply early stopping
early_stopping_epoch = 20

# Early stopping variables
best_loss = 10000000
best_epoch = 0
early_stopped = False

# Prepare our training dataset with batch
dataset_batch = tf.contrib.data.Dataset.from_tensor_slices((X_train2, y_train2)).batch(batch_size).repeat(training_epochs)
dataset_batch = dataset_batch.make_initializable_iterator()


with tf.Session() as sess:
    # initialize
    sess.run(init_g)
    sess.run(dataset_batch.initializer)
    
    # restore our model
    restore_saver.restore(sess, "./hw2_model/Team20_HW2.ckpt")
    for var in output_layer:
        var.initializer.run()

    start_time = time.time()
    
    # Training 1000 epochs
    for epoch in range(0, training_epochs):
        # Training steps
        for i in range(train_steps):
            X_in, y_in = sess.run(dataset_batch.get_next())
            sess.run(training_op, feed_dict={x: X_in, y: y_in, training_mode: False})

        # Validate accuracy every epoch
        curr_loss, curr_accuracy = sess.run([loss, accuracy], feed_dict={x: X_valid2, y: y_valid2, training_mode: False})
        
        # Save checkpoint of current model if it performs better
        if best_loss > curr_loss:
            best_loss = curr_loss
            save_path = saver.save(sess, "./Team20_HW3_4_bonus.ckpt")
            best_epoch = epoch        
        # Early stop if model does not improve for certain epoch
        elif epoch - best_epoch >= early_stopping_epoch:
            early_stopped = True
            break
        
        print("Epoch {}: Validation loss: {} Best loss: {} Accuracy: {} ".format(epoch, curr_loss, best_loss ,curr_accuracy))

    # Save checkpoint in case the training is not early-stopped
    if not early_stopped:
        print("save best model")
        save_path = saver.save(sess, "./Team20_HW3_4_bonus.ckpt")

    end_time = time.time()
    print("Total training time: " + str(end_time - start_time))    
    
    # Get the best model
    saver.restore(sess, "./Team20_HW3_4_bonus.ckpt")
    
    # Total accuracy
    final_accuracy = sess.run(accuracy, feed_dict={x: X_test2, y: y_test2, training_mode: False})
    print("Test accuracy: ", final_accuracy)


INFO:tensorflow:Restoring parameters from ./hw2_model/Team20_HW2.ckpt
Epoch 0: Validation loss: 1.3878096342086792 Best loss: 1.3878096342086792 Accuracy: 0.4399999976158142 
Epoch 1: Validation loss: 0.8927179574966431 Best loss: 0.8927179574966431 Accuracy: 0.699999988079071 
Epoch 2: Validation loss: 0.7505435943603516 Best loss: 0.7505435943603516 Accuracy: 0.7133333086967468 
Epoch 3: Validation loss: 0.6249088048934937 Best loss: 0.6249088048934937 Accuracy: 0.800000011920929 
Epoch 4: Validation loss: 0.528852641582489 Best loss: 0.528852641582489 Accuracy: 0.8466666340827942 
Epoch 5: Validation loss: 0.477429062128067 Best loss: 0.477429062128067 Accuracy: 0.846666693687439 
Epoch 6: Validation loss: 0.43718814849853516 Best loss: 0.43718814849853516 Accuracy: 0.8533333539962769 
Epoch 7: Validation loss: 0.41405147314071655 Best loss: 0.41405147314071655 Accuracy: 0.8866666555404663 
Epoch 8: Validation loss: 0.4017956554889679 Best loss: 0.4017956554889679 Accuracy: 0.886666

#### Test accuracy of HW3.3 is 0.851882
#### Test accuracy of HW3.4 is 0.902901
### Accuracy of HW3.4 is better than HW3.3